In [1]:
print("Hello world")

Hello world


In [24]:
import tweepy
from deep_translator import GoogleTranslator
token = "" #bearer token, toegang als developer
client = tweepy.Client(bearer_token=token)

userid = "7174972" #twitter id, converter
paginator = tweepy.Paginator(client.get_users_tweets, id=userid, max_results=8) #https://docs.tweepy.org/en/stable/client.html
translated = GoogleTranslator(source='auto', target='es').translate(text="Laatste tweets van de NOS:")
print(translated)

for tweet in paginator.flatten(limit=8): #Acht resultaten
    #tweet casten naar een string
    translateTweets = GoogleTranslator(source='auto', target='en').translate(str(tweet))
    print(translateTweets)
    print(tweet, "\n")


Últimos tweets de NOS:
Chinese embassy in The Hague defaced with red paint during demonstration https://t.co/kgN1kiwiDQ
Chinese ambassade Den Haag bij betoging beklad met rode verf https://t.co/kgN1kiwiDQ 

Freight train with many cars on fire at Etten-Leur https://t.co/DTHKubnnQ9
Goederentrein met veel auto's in brand bij Etten-Leur https://t.co/DTHKubnnQ9 

Defeat for online supers: staff should get weekend allowance https://t.co/vRtPVxCyox
Nederlaag voor online supers: personeel moet weekendtoeslag krijgen https://t.co/vRtPVxCyox 

Woman raids Beirut bank to withdraw savings for sick sister https://t.co/5qldx5RxAa
Vrouw valt bank in Beiroet binnen om spaargeld voor zieke zus op te nemen https://t.co/5qldx5RxAa 

Cardiologists get millions of euros from medical companies without permission https://t.co/KI9z8j8YUO
Cardiologen krijgen zonder toestemming miljoenen euro's van medische bedrijven https://t.co/KI9z8j8YUO 

'Jumbo CEO Van Eerd fixed as a suspect in money laundering' https://